<a href="https://colab.research.google.com/github/A190nux/Anomaly-detection/blob/main/Norm_Cut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import train_test_split
import pickle

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Extracting the features

In [3]:
featuresPath = "/content/drive/MyDrive/Colab Notebooks/Anomaly Detection/features.txt"
with open(featuresPath, "r") as file:
  # splitting at ':' and taking the first word
  features = [line.split(':')[0] for line in file]
# dropping the first 2 lines as they are unnecessary
features.pop(0), features.pop(0)
features.append('attack_type')

In [4]:
len(features)

41

## Reading the dataset

In [5]:
path = "/content/drive/MyDrive/Colab Notebooks/Anomaly Detection/kddcup.data_10_percent.gz"
df = pd.read_csv(path, compression='gzip', names=features, header=None)
df.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,tcp,http,SF,181,5450,0,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,239,486,0,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,235,1337,0,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,219,1337,0,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,217,2032,0,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


## Changing the categorical features to numerical

In [6]:
# getting all categorical columns
cat_columns = df.select_dtypes(['object']).columns
# converting all categorical columns to numeric
df[cat_columns] = df[cat_columns].apply(lambda x: pd.factorize(x)[0])
df.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,0,0,0,181,5450,0,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,0
0,0,0,0,239,486,0,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0
0,0,0,0,235,1337,0,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0
0,0,0,0,219,1337,0,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0
0,0,0,0,217,2032,0,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0


In [7]:
y = df.iloc[:,-1:].to_numpy()
x = df.iloc[:,:-1].to_numpy()
print(y.shape)
print(x.shape)

(494021, 1)
(494021, 40)


In [8]:
df.describe()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
count,494021.000000,494021.000000,494021.000000,4.940210e+05,4.940210e+05,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,...,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000
mean,1.189338,8.392376,0.835960,3.025610e+03,8.685324e+02,0.000045,0.006433,0.000014,0.034519,0.000152,...,188.665670,0.753780,0.030906,0.601935,0.006684,0.176754,0.176443,0.058118,0.057412,3.932560
std,0.960704,4.973796,1.579212,9.882181e+05,3.304000e+04,0.006673,0.134805,0.005510,0.782103,0.015520,...,106.040437,0.410781,0.109259,0.481309,0.042133,0.380593,0.380919,0.230590,0.230140,2.297779
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,9.000000,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,2.000000,9.000000,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
75%,2.000000,10.000000,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
max,2.000000,65.000000,10.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,22.000000


In [24]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.005, random_state=42, stratify=y)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2470, 40)
(491551, 40)
(2470, 1)
(491551, 1)


In [26]:
np.unique(y_train)

array([ 0,  4,  5,  7,  8,  9, 10, 13, 15, 17, 20])

In [12]:
file_name = 'X_train.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(X_train, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "X_train.pkl"


In [13]:
file_name = 'X_test.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(X_test, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "X_test.pkl"


In [14]:
file_name = 'y_train.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(y_train, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "y_train.pkl"


In [15]:
file_name = 'y_test.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(y_test, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "y_test.pkl"


In [16]:
file_name = 'X_train.pkl'
with open(file_name, 'rb') as file:
    X_train = pickle.load(file)
    print(f'Object successfully loaded from "{file_name}"')

Object successfully loaded from "X_train.pkl"


In [17]:
file_name = 'X_test.pkl'
with open(file_name, 'rb') as file:
    X_test = pickle.load(file)
    print(f'Object successfully loaded from "{file_name}"')

Object successfully loaded from "X_test.pkl"


In [18]:
file_name = 'y_train.pkl'
with open(file_name, 'rb') as file:
    y_train = pickle.load(file)
    print(f'Object successfully loaded from "{file_name}"')

Object successfully loaded from "y_train.pkl"


In [19]:
file_name = 'y_test.pkl'
with open(file_name, 'rb') as file:
    y_test = pickle.load(file)
    print(f'Object successfully loaded from "{file_name}"')

Object successfully loaded from "y_test.pkl"


In [20]:
def normcut(F,k,gamma,kernel = True):
    if kernel == True:
        kern = rbf_kernel(F, gamma=gamma)
    else:
        kern = kneighbors_graph(F,k).toarray()
    kern = np.array(kern)
    degree = np.zeros(len(kern))
    c_sum = kern.sum(axis = 0)
    r_sum = kern.sum(axis = 1)
    for i in range(c_sum.shape[0]):
        degree[i] = (c_sum[i] + r_sum[i])/2
    degree = np.diag(degree)
    kern = degree - kern
    degree = np.linalg.inv(degree)
    kern = np.matmul(degree,kern)
    (evalues,evectors) = np.linalg.eig(kern)
    idx = evalues.argsort()
    evalues = [idx]
    evectors = evectors[ : , idx]
    evectors = evectors.T[:k]
    normFactor = np.sqrt(np.sum(np.power(evectors,2),axis=1))
    normFactor = normFactor.reshape((len(evectors),1))
    for i in range(len(normFactor)):
        if normFactor[i] == 0:
            normFactor[i] = 1
    Y = np.real(evectors/normFactor).T
    return Y, KMeans(n_clusters=k).fit(Y).labels_

In [ ]:
(Y_h , labels_h) = normcut(X_train,23,0.01,kernel = False)

In [ ]:
Y_h.shape

In [ ]:
labels_h.shape

In [ ]:
labels_h.max()

In [ ]:
y_train.max()